In [56]:
import pickle
with open("outputs/sst2/lstm/modifications_0.pickle", 'rb') as file:
    modifications = pickle.load(file)

with open("train_instances.pickle", 'rb') as file:
    train_instances = pickle.load(file)



In [57]:
i = 11
pos = int(list(modifications[i].keys())[0])
print(modifications[i], train_instances[i]["tokens"].tokens[pos], train_instances[i]["tokens"].tokens)

{13: 'stiflingly'} sequences [Still, ,, this, flick, is, fun, ,, and, host, to, some, truly, excellent, sequences, .]
